# Predicting Omega
Given a trained network use this document to predict the values of Omega to produce a desired $J_{i,j}$.


In [ ]:
import sys
sys.path.insert(0, '')
import classesmu
from classesmu import BatchSimulatedSpinLattice as bsslmu
from classesmu import SimulatedSpinLattice as sslmu
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pickle
import time
import ic_functions
from ic_functions import chain, circle, torch_normalized, np_normalized, noise_adder
import neural_nets
from neural_nets import Net
import matplotlib.pyplot as plt
import os
from mpl_toolkits.mplot3d import Axes3D

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Instructions
Enter the path of the saved model to the path variable.

In [ ]:
path = ''
d = torch.load(path)
"""The required items from the file are stored as variables."""
ic = d['ic']
N=ic.n

mu=d['mu']
m=d['m']

input_size = d['input_size']
hidden_size = d['hidden_size']
output_size = d['output_size']

"""The trained model is loaded"""
model = Net(input_size, hidden_size, output_size,1)
model.load_state_dict(d['model_state_dict'])
optimizer = torch.optim.Adam(model.parameters())
optimizer.load_state_dict(d['optimizer_state_dict'])
for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.double()

model.double()



## Instructions C'td
Enter the desired $J_{i,j}$ value or set of values in the variable Jij. Note make sure the values are normalized by dividing by the L2-norm.

In [ ]:
"""Network Predictions"""
Jij = chain(N)
Jij = torch.from_numpy(np_normalized(Jij))
if len(Jij.size())==1:
    Omega_pred = model(Jij).view(1,N,m)
else:
    Omega_pred = model(Jij).view(Jij.size()[0],N,m)
lattice = bsslmu(ic, mu, Omega_pred, dev=device)
Jij_pred = lattice.normalize()
Error = torch.mean(torch.abs(Jij-Jij_pred))*100
print('Predicted Omega values\n', Omega_pred)
print('Desired Jijs\n',Jij)
print('Jijs give by Omegas predicted\n', Jij_pred)
print('Mean error\n', Error.item())

In [ ]:
"""A bar graph to help one visalize the scales of the desired and original Jijs for a single Jij"""
if len(Jij.size())==1:
    size_Jij = int(N*(N-1)/2)
    x = range(1,size_Jij+1)

    plt.figure(figsize=(size_Jij/4,5))

    barWidth = 0.25
 
    bars1 = list(Jij.numpy())
    bars2 = list(Jij_pred.view(size_Jij).detach().numpy())

 
    
    r1 = [i - barWidth/2 for i in x]
    r2 = [i + barWidth/2 for i in x]


 
   
    plt.bar(r1, bars1, width=barWidth, edgecolor='white', label='Original Jij')
    plt.bar(r2, bars2, width=barWidth, edgecolor='white', label='Predicted Jij')

 
    
    plt.xlabel('Entry of Vectorized Jij', fontweight='bold')
    plt.ylabel('Normalized Jij', fontweight='bold')
    plt.title('N=13 Triangular Lattice', fontweight='bold')
 
    
    plt.legend()
    plt.show()


In [ ]:
"""A comparision of the absolute sizes of the Rabi frequencies corresponding to each detuning. Used to predict which
mu value may be possible to eliminate and still retain some accuracy."""
numerator=torch.einsum('pnm->pm',[abs(Omega_pred)])
denominator=torch.einsum('pnm->p',[abs(Omega_pred)])
epsilon=torch.einsum('pm,p->m',[numerator,1/denominator])
norm_epsilon=torch.sqrt((epsilon ** 2).sum(0))
normalized_epsilon=epsilon/norm_epsilon

In [ ]:
"""Visualization of the above calculations"""
plt.plot(range(1,len(epsilon)+1),list(normalized_epsilon),
            marker='o',linestyle='None')

plt.ylabel('Mu Usage')
plt.xlabel('m')
plt.title('Scaled value')

In [ ]:
"""Determine which indices of the original mu were removed, incase the user wishes to remove more, and produce a
new data set."""
# print(d['removed_mus'])
# print(d['mu'])